In [1]:
import sys
assert sys.version_info >= (3, 5)

import re,os,csv
import numpy as np
import pandas as pd
from itertools import chain

#### RAKE STUFF

In [5]:
RAKE_data_dir = './rake_output'

In [6]:
rake_files =  ['ClinTrials_RAKE.csv','NIH_RAKE.csv','PubMed_RAKE.csv']

In [7]:
fg = pd.read_csv('rake_output/PubMed_RAKE.csv')

In [8]:
def join_kw_rake_cols(fileName):
    df = pd.read_csv(fileName)
    kw_cols = [x  for x in df.columns.tolist() if x.startswith('KW_RAKE')]
    df['Keywords'] = df[kw_cols[0]]
    for i in range(1,len(kw_cols)): 
        df['Keywords'] =  df['Keywords'] + df[kw_cols[i]]
        df['Keywords'] =  df.Keywords.str.replace('\]\[' , ',',regex=False)
    df['Keywords'] =  df.Keywords.str.replace('\]|\[' , '',regex=True)
    df.drop([x  for x in df.columns.tolist() if x.startswith('Unnamed')],axis=1,inplace=True)
    df.drop(kw_cols,axis=1,inplace=True)
    df.to_csv('preProc_' + os.path.basename(fileName),index=False)
    return 'preProc_' + os.path.basename(fileName)

In [10]:
preProc_RAKE_files = [join_kw_rake_cols(RAKE_data_dir + '/' + f) for f in rake_files]    

In [11]:
preProc_RAKE_files

['preProc_ClinTrials_RAKE.csv',
 'preProc_NIH_RAKE.csv',
 'preProc_PubMed_RAKE.csv']

#### AUTOPHRASE STUFF

In [12]:
AUTOP_data_dir = './Auto_phrase/'

In [13]:
autop_files =  ['clinical_data_output_withkeyword.csv','NIH_data_output_withkeyword.csv','pubmed_data_output_withkeyword.csv']

In [ ]:
#### Bad files, cant be used. Ravi to update and add cleaned up files

#### LDA STUFF

In [ ]:
LDA_data_dir = './new_data/Data_with_keywords/Data_with_keywords/LDA/'

In [ ]:
autop_files =  ['clinicTrial_LDA_cat_output.csv','pubMed_LDA_cat_output.csv']

#### File Map across 3

In [ ]:
allData_dict_PTR = {'pubmed' : ['preProc_PubMed_RAKE.csv','pubMed_LDA_cat_output.csv'],
                   'clinictrial' : ['preProc_ClinTrials_RAKE.csv','clinicTrial_LDA_cat_output.csv'],
                   'nih': ['preProc_NIH_RAKE.csv']}

#### Merge keywords across 3

In [ ]:
if not os.path.exists('CONSOLIDATED_FOR_ENTITY_MATCHING'):
    os.makedirs('CONSOLIDATED_FOR_ENTITY_MATCHING')
    
for k in allData_dict_PTR.keys():
    df_1 = pd.read_csv(allData_dict_PTR[k][0])
    print(f'[INFO] consolidating for : {k}')
    for n in range(1,(len(allData_dict_PTR[k]))):
        df_1 = df_1.merge(pd.read_csv(allData_dict_PTR[k][n])[['Title','Keywords']], left_on=['Title'],right_on=['Title'])
        df_1['Keywords'] = df_1.Keywords_x + df_1.Keywords_y
        df_1['Keywords'] = df_1.Keywords.str.replace('\'','').str.split(' ').apply(lambda x : list(set(x)))
        df_1.drop(['Keywords_x','Keywords_y'],axis=1,inplace=True)
    df_1.to_csv('CONSOLIDATED_FOR_ENTITY_MATCHING/' + 'all_' + k + '_keyWords_FINAL.csv',index=False)    